# Outline of Data Cleaning Process 🧽:

## Data Plan
- The IMDB dataset will act as the knowledge base for the top 1000 movies I will be analyzing. Data from other sources will subsequently be matched up with these entries.
- Core Goal: Gather up everything i one place before transforming it into a proper database with key relations defined.

## Contents of This Notebook
- Documenting assumptions and normalization done for the data
- Preliminary data inspection

**Notes on IMDB Data:**
- Understanding **IMDB Rating**
  - IMDB rating is actually weighted by a variety of factors including a voters overall voting reputation in the past (https://www.getafollower.com/blog/imdb-ratings/)
  - This is what makes it a particularly reliable metric when it comes to understanding a movie's success
- Understanding **Certificate**
  - link to certificate documentation: https://help.imdb.com/article/contribution/titles/certificates/GU757M8ZJ9ZPXB39#
- Understanding **Gross**
  - Gross box office earnings for movie
  - Measured in straight USD

**Notes on Rotten Tomatoes Data:**
- Raw form splits reviews and movie overview in separate datasets
- Goal: retain necessary data and join with IMDB data in a single dataframe
    - Want to match up by movie, year, and maybe director for extra assurance --> should filter to only include the top 1000 movies at this point
    - Once all cleaned data together, a relational database schema can be developed and constructed for the data

# 1. Handling Imports and Data Access

In [6]:
# Required library imports
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import numpy as np

In [23]:
# Accessing IMDB data from Kaggle and putting into a DataFrame

# Set the path to the file you'd like to load
file_path = "imdb_top_1000.csv"
movies_tomatoes_path = "Raw/rotten_tomatoes_movies.csv"
reviews_filepath = "Raw/rotten_tomatoes_critic_reviews.csv"

# Load the latest version
movies = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows",
  file_path
)

tomato_movies = df = pd.read_csv(movies_tomatoes_path)
tomato_reviews = pd.read_csv(reviews_filepath)

tomato_reviews.head()



C:\Users\milen\AppData\Local\Temp\ipykernel_57024\1812086170.py:9: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  movies = kagglehub.load_dataset(


,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...


# 2. Inspecting Data and Beginning Data Cleaning Process

**Attributes Contained in Original IMDB Data:**
- Poster_Link
- Series_Title
- Released_Year
- Certificate (age rating)
- Runtime
- Genre
- IMDB_Rating (score of movie from IMDB; seems to be out of 10)
- Overview
- Meta_score (score for movie; seems to be out of 100)
- Director
- Star1
- Star2
- Star3
- Star4 --- Star1 - Star4 = top 4 star actors for the movie
- No_of_Votes
- Gross -- money made by movie

**Attributes Contained in Original Rotten Tomatoes Data:**
- **Movies Dataset:**
    - rotten_tomatoes_link ⭐
    - movie_title ⭐
    - movie_info
    - critics_consensus ⭐
    - content_rating
    - genres
    - directors ⭐
    - authors
    - actors
    - original_release_date ⭐
    - streaming_release_date
    - runtime
    - production_company
    - tomatometer_status ⭐
    - tomatometer_rating ⭐
    - tomatometer_count ⭐
    - audience_status ⭐
    - audience_rating ⭐
    - audience_count ⭐
    - tomatometer_top_critics_count ⭐
    - tomatometer_fresh_critics_count ⭐
    - tomatometer_rotten_critics_count ⭐

In [24]:
# looking at the columns

movies.columns
movies.head()

tomato_movies.columns


Index(['rotten_tomatoes_link', 'movie_title', 'movie_info',
       'critics_consensus', 'content_rating', 'genres', 'directors', 'authors',
       'actors', 'original_release_date', 'streaming_release_date', 'runtime',
       'production_company', 'tomatometer_status', 'tomatometer_rating',
       'tomatometer_count', 'audience_status', 'audience_rating',
       'audience_count', 'tomatometer_top_critics_count',
       'tomatometer_fresh_critics_count', 'tomatometer_rotten_critics_count'],
      dtype='object')

**Attributes I will discard:** The poster link is not useful for this project so I will get rid of it and keep everything else for now

**TO-DO For Cleaning:**
- Discarding 
- Adjusting types appropriately
  - Make sure to handle commas in large numerals!
  - year, runtime, gross
- Getting rid of weird duration formating: 'time min' --> time
- Handling null values:
  - only unacceptable null value in this use-case would be for title (not an issue)
  - Just mark them consistently with None

In [14]:
movies.dtypes

Poster_Link          str
Series_Title         str
Released_Year        str
Certificate          str
Runtime              str
Genre                str
IMDB_Rating      float64
Overview             str
Meta_score       float64
Director             str
Star1                str
Star2                str
Star3                str
Star4                str
No_of_Votes        int64
Gross                str
dtype: object

# Temp Data Cleaning Code

In [13]:
# Function to clean imdb dataframe
def data_cleaning_imdb(df) -> None:
  # Dropping the poster link column
  df = df.drop("Poster_Link", axis=1)

  # Replace string "NaN" with real NaN
  df = df.replace("NaN", np.nan)

  # Clean Runtime column safely
  df["Runtime"] = pd.to_numeric(
      df["Runtime"].astype(str).str.replace(" min", "").str.strip(),
      errors="coerce"
  )

  # Clean Gross column safely
  df["Gross"] = pd.to_numeric(
      df["Gross"].astype(str).str.replace(",", ""),
      errors="coerce"
  ).astype("Int64")

  # Clean Released Year safely
  df["Released_Year"] = pd.to_numeric(
      df["Released_Year"],
      errors="coerce"
  ).astype("Int64")

  # Renaming cols to simpler versions in my preferred format
  df.rename(columns={
    "Series_Title": "title",
    "Released_Year": "year",
    "Runtime": "runtime",
    "Genre": "genre",
    "IMDB_Rating": "imdb_rating",
    "Meta_score": "meta_score",
    "Director": "director",
    "No_of_Votes": "vote_count_imdb",
    "Gross": "box_office"

}, inplace=True)
  return df

imdb_movies = data_cleaning_imdb(movies)
imdb_movies

,title,year,Certificate,runtime,genre,imdb_rating,Overview,meta_score,director,Star1,Star2,Star3,Star4,vote_count_imdb,box_office
0,The Shawshank Redemption,1994,A,142,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,28341469
1,The Godfather,1972,A,175,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,134966411
2,The Dark Knight,2008,UA,152,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,534858444
3,The Godfather: Part II,1974,A,202,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,57300000
4,12 Angry Men,1957,U,96,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,4360000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Breakfast at Tiffany's,1961,A,115,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,166544,<NA>
996,Giant,1956,G,201,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,34075,<NA>
997,From Here to Eternity,1953,Passed,118,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,43374,30500000
998,Lifeboat,1944,NaN,97,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,<NA>
